In [1]:
import pandas as pd
import os
import glob
from datetime import datetime
import seaborn as sns
from sklearn.metrics import mean_squared_log_error
from sklearn.ensemble import RandomForestClassifier


In [2]:
path = '/Users/ishantchaudhary/DL_Assignment/Crypto_data_minute/'
extension = 'txt'
os.chdir(path)
result = glob.glob('*.{}'.format(extension))

In [11]:
#Appending the dataset of different cryptos with necessary transformations
dfs = list()
tr_dfs = list()
tst_dfs = list()
for f in result:
#     data = pd.read_csv(f)
    data = pd.read_csv(f,header = None)

    size = len(f)
    data['Crypto'] = f[:size - 9]
    data.columns = [ 'DateTime','Open', 'High', 'Low', 'Close', 'Volume','Crypto']
    data['DateTime']= pd.to_datetime(data['DateTime'])
    data = data.resample('H', on='DateTime').max()
    data['DateTime'] = data.index
    # adding percentage change in closing price 
    data['Change_close'] = data['Close'].pct_change()
    data['Change_close'] = data['Change_close'].bfill()
#   data['true_labels'] = (data['Change_close'] > 0).astype(int)
    # feature engineering basic 
    data['Close t-1'] = data['Close'].shift()
    data['Close t-1'] = data['Close t-1'].bfill()
    data['Change_close t-1'] = data['Change_close'].shift()
    data['Change_close t-1'] = data['Change_close t-1'].bfill()
    
    # add true labels on basis of change close 
    data['true_labels'] = (data['Change_close'] > 0).astype(int)
    
    data['naive_pred'] = data['true_labels'].shift()
    data['naive_pred'] = data['naive_pred'].bfill()
    # getting train and test
    training_data = data[data.DateTime<datetime(2021, 9, 30, 23)]
    test_data = data[data.DateTime>=datetime(2021, 9, 30, 23)]
    
    
    dfs.append(data)
    tr_dfs.append(training_data)
    tst_dfs.append(test_data)

df = pd.concat(dfs, ignore_index=True)
train_df = pd.concat(tr_dfs, ignore_index=True)
test_df = pd.concat(tst_dfs, ignore_index=True)
# df.columns = [ 'DateTime','Open', 'High', 'Low', 'Close', 'Volume','Crypto']

 

In [12]:
df

,DateTime,Open,High,Low,Close,Volume,Crypto,Change_close,Close t-1,Change_close t-1,true_labels,naive_pred
0,2018-03-01 00:00:00,0.324,0.324,0.324,0.324,600.000000,ADA,0.000000,0.324,0.000000,0,0.0
1,2018-03-01 01:00:00,0.324,0.324,0.324,0.324,7000.000000,ADA,0.000000,0.324,0.000000,0,0.0
2,2018-03-01 02:00:00,0.316,0.316,0.316,0.316,900.000000,ADA,-0.024691,0.324,0.000000,0,0.0
3,2018-03-01 03:00:00,0.316,0.316,0.316,0.316,2400.000000,ADA,0.000000,0.316,-0.024691,0,0.0
4,2018-03-01 04:00:00,0.310,0.310,0.310,0.310,5000.000000,ADA,-0.018987,0.316,0.000000,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
503509,2022-09-30 19:00:00,27.970,27.990,27.940,27.970,1171.793105,ETC,-0.002781,28.048,0.001357,0,1.0
503510,2022-09-30 20:00:00,27.754,27.810,27.730,27.760,659.071936,ETC,-0.007508,27.970,-0.002781,0,0.0
503511,2022-09-30 21:00:00,27.699,27.713,27.699,27.699,918.180473,ETC,-0.002197,27.760,-0.007508,0,0.0
503512,2022-09-30 22:00:00,27.626,27.641,27.619,27.641,1104.017960,ETC,-0.002094,27.699,-0.002197,0,0.0


In [13]:
train_df.head(30)

,DateTime,Open,High,Low,Close,Volume,Crypto,Change_close,Close t-1,Change_close t-1,true_labels,naive_pred
0,2018-03-01 00:00:00,0.324,0.324,0.324,0.324,600.0,ADA,0.000000,0.324,0.000000,0,0.0
1,2018-03-01 01:00:00,0.324,0.324,0.324,0.324,7000.0,ADA,0.000000,0.324,0.000000,0,0.0
2,2018-03-01 02:00:00,0.316,0.316,0.316,0.316,900.0,ADA,-0.024691,0.324,0.000000,0,0.0
3,2018-03-01 03:00:00,0.316,0.316,0.316,0.316,2400.0,ADA,0.000000,0.316,-0.024691,0,0.0
4,2018-03-01 04:00:00,0.310,0.310,0.310,0.310,5000.0,ADA,-0.018987,0.316,0.000000,0,0.0
5,2018-03-01 05:00:00,0.308,0.310,0.308,0.309,5000.0,ADA,-0.003226,0.310,-0.018987,0,0.0
6,2018-03-01 06:00:00,0.310,0.310,0.310,0.310,2100.0,ADA,0.003236,0.309,-0.003226,1,0.0
7,2018-03-01 07:00:00,0.324,0.324,0.324,0.324,3500.0,ADA,0.045161,0.310,0.003236,1,1.0
8,2018-03-01 08:00:00,0.324,0.324,0.324,0.324,1300.0,ADA,0.000000,0.324,0.045161,0,1.0
9,2018-03-01 09:00:00,0.315,0.315,0.315,0.315,1200.0,ADA,-0.027778,0.324,0.000000,0,0.0


In [14]:
test_df

,DateTime,Open,High,Low,Close,Volume,Crypto,Change_close,Close t-1,Change_close t-1,true_labels,naive_pred
0,2021-09-30 23:00:00,2.117,2.117,2.116,2.117,352562.445000,ADA,-0.004233,2.126,0.005201,0,1.0
1,2021-10-01 00:00:00,2.133,2.135,2.131,2.133,457432.312000,ADA,0.007558,2.117,-0.004233,1,0.0
2,2021-10-01 01:00:00,2.128,2.130,2.126,2.128,166378.194000,ADA,-0.002344,2.133,0.007558,0,1.0
3,2021-10-01 02:00:00,2.126,2.128,2.125,2.126,298618.621000,ADA,-0.000940,2.128,-0.002344,0,0.0
4,2021-10-01 03:00:00,2.107,2.108,2.105,2.107,282851.443000,ADA,-0.008937,2.126,-0.000940,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
87605,2022-09-30 19:00:00,27.970,27.990,27.940,27.970,1171.793105,ETC,-0.002781,28.048,0.001357,0,1.0
87606,2022-09-30 20:00:00,27.754,27.810,27.730,27.760,659.071936,ETC,-0.007508,27.970,-0.002781,0,0.0
87607,2022-09-30 21:00:00,27.699,27.713,27.699,27.699,918.180473,ETC,-0.002197,27.760,-0.007508,0,0.0
87608,2022-09-30 22:00:00,27.626,27.641,27.619,27.641,1104.017960,ETC,-0.002094,27.699,-0.002197,0,0.0


In [7]:
train_df.drop(['Open', 'High', 'Low'], axis=1,inplace =True)
test_df.drop(['Open', 'High', 'Low'], axis=1,inplace =True)

In [8]:
train_df.sort_values(by=['DateTime'],inplace=True)
train_df.reset_index(inplace=True)
train_df = train_df.iloc[: , 1:]

In [9]:
test_df.sort_values(by=['DateTime'],inplace=True)
test_df.reset_index(inplace=True)
test_df = test_df.iloc[: , 1:]

In [17]:
train_df

,DateTime,Close,Volume,Crypto,Change_close,Close t-1,Change_close t-1,true_labels
0,2013-04-01 00:00:00,93.155,12.250,BTC,0.006817,93.155,0.006817,1
1,2013-04-01 01:00:00,93.790,24.720,BTC,0.006817,93.155,0.006817,1
2,2013-04-01 02:00:00,94.480,122.500,BTC,0.007357,93.790,0.006817,1
3,2013-04-01 03:00:00,NaN,NaN,NaN,0.000000,94.480,0.007357,0
4,2013-04-01 04:00:00,94.000,4.900,BTC,-0.005080,94.000,0.000000,0
...,...,...,...,...,...,...,...,...
415899,2021-09-30 22:00:00,0.953,147002.131,XRP,0.001050,0.952,0.004219,1
415900,2021-09-30 22:00:00,43896.900,42.559,BTC,0.003280,43753.380,0.000135,1
415901,2021-09-30 22:00:00,253.270,418.207,XMR,-0.000355,253.360,0.005716,0
415902,2021-09-30 22:00:00,2.126,335594.032,ADA,0.005201,2.115,0.001420,1


In [111]:
print(test_df.head(10))

             DateTime      Close      Volume Crypto  Change_close  Close t-1  \
0 2021-09-30 23:00:00      2.117  352562.445    ADA     -0.004233      2.126   
1 2021-09-30 23:00:00      0.954  153540.311    XRP      0.001049      0.953   
2 2021-09-30 23:00:00     24.028   29523.623   LINK      0.008267     23.831   
3 2021-09-30 23:00:00    153.500     630.377    LTC     -0.000065    153.510   
4 2021-09-30 23:00:00    251.370     186.040    XMR     -0.007502    253.270   
5 2021-09-30 23:00:00   3010.210     417.078    ETH     -0.003562   3020.970   
6 2021-09-30 23:00:00  43899.000      34.925    BTC      0.000048  43896.900   
7 2021-09-30 23:00:00     47.044     829.679    ETC     -0.000340     47.060   
8 2021-09-30 23:00:00      0.279  539000.594    XLM      0.003597      0.278   
9 2021-09-30 23:00:00      0.089   22233.487    TRX      0.000000      0.089   

   Change_close t-1  true_labels  
0          0.005201            0  
1          0.001050            1  
2          0.0

In [12]:
# Checking Start and End date for each crypto and number of entries
# df.groupby(['Crypto']).agg(['first', 'last','count']).stack()['DateTime']

In [13]:
# df.groupby(['Crypto']).agg(['first']).stack()['DateTime']

In [14]:
# df.groupby(['Crypto']).corr()

In [15]:
# df.to_csv("pooled_data_hrly")

In [16]:
# next is to split using the time line so that all cryptos have same time line 

In [27]:
train_df=train_df.dropna()
test_df=test_df.dropna()

In [28]:
train_df.isna().sum()

DateTime            0
Close               0
Volume              0
Crypto              0
Change_close        0
Close t-1           0
Change_close t-1    0
true_labels         0
dtype: int64

In [29]:
test_df.isna().sum()

DateTime            0
Close               0
Volume              0
Crypto              0
Change_close        0
Close t-1           0
Change_close t-1    0
true_labels         0
dtype: int64

In [77]:
from sklearn.preprocessing import LabelEncoder

LE = LabelEncoder()

In [78]:
xtr, xts = train_df.drop(['Close','Volume','Change_close','true_labels'], axis=1), test_df.drop(['Volume','Change_close','Close','true_labels'], axis=1)
# xtr['DateTime'], xts['DateTime'] = xtr['DateTime'].astype(str), xts['DateTime'].astype(str)

xtr.index,xts.index = xtr['DateTime'], xts['DateTime']
xtr, xts = xtr.drop(['DateTime'], axis=1), xts.drop(['DateTime'], axis=1)

xtr['Crypto'],xts['Crypto'] = LE.fit_transform(xtr['Crypto']),LE.fit_transform(xts['Crypto'])
ytr, yts = train_df['true_labels'].values, test_df['true_labels'].values


In [115]:
xts.head(30)

,Crypto,Close t-1,Change_close t-1
DateTime,,,
2021-09-30 23:00:00,0,2.126,0.005201
2021-09-30 23:00:00,9,0.953,0.001050
2021-09-30 23:00:00,4,23.831,0.004045
2021-09-30 23:00:00,5,153.510,0.005173
2021-09-30 23:00:00,8,253.270,-0.000355
2021-09-30 23:00:00,3,3020.970,0.004332
2021-09-30 23:00:00,1,43896.900,0.003280
2021-09-30 23:00:00,2,47.060,0.004075
2021-09-30 23:00:00,7,0.278,0.003610


In [137]:
xts2 = xts.loc['2021-09-30 23:00:00']
size = xts2.shape[0]
yts2 = yts[0:size]

In [149]:

xts3 = xts.loc['2021-09-30 23:00:00':'2021-10-01 01:00:00']
size = xts3.shape[0]
yts3 = yts[0:size]

In [ ]:
2021-09-30 23:00:00
        

In [150]:
xts3

,Crypto,Close t-1,Change_close t-1
DateTime,,,
2021-09-30 23:00:00,0,2.126,0.005201
2021-09-30 23:00:00,9,0.953,0.001050
2021-09-30 23:00:00,4,23.831,0.004045
2021-09-30 23:00:00,5,153.510,0.005173
2021-09-30 23:00:00,8,253.270,-0.000355
2021-09-30 23:00:00,3,3020.970,0.004332
2021-09-30 23:00:00,1,43896.900,0.003280
2021-09-30 23:00:00,2,47.060,0.004075
2021-09-30 23:00:00,7,0.278,0.003610


In [164]:
xts4 = xts.loc['2021-09-30 23:00:00':'2021-10-30 23:00:00']
size = xts4.shape[0]
yts4 = yts[0:size]

In [181]:
test_df.tail()

,DateTime,Close,Volume,Crypto,Change_close,Close t-1,Change_close t-1,true_labels
87605,2022-09-30 23:00:00,53.56000,9.634356e+02,LTC,0.002733,53.4140,0.000637,1
87606,2022-09-30 23:00:00,0.06113,5.833922e+03,TRX,0.002131,0.0610,-0.001800,1
87607,2022-09-30 23:00:00,0.43560,1.955616e+06,ADA,0.005772,0.4331,0.003941,1
87608,2022-09-30 23:00:00,1329.09000,4.809121e+02,ETH,0.001741,1326.7800,-0.002998,1
87609,2022-09-30 23:00:00,27.76000,1.875081e+03,ETC,0.004305,27.6410,-0.002094,1


In [182]:

        
xts5 = xts.loc['2021-09-30 23:00:00':'2022-09-30 23:00:00']
size = xts5.shape[0]
yts5 = yts[0:size]

In [54]:
# clf = RandomForestClassifier(n_estimators=1000, n_jobs=-1, random_state=0)
# # clf.fit(xtr, ytr)

RandomForestClassifier(n_estimators=1000, n_jobs=-1, random_state=0)

In [55]:
# pred = clf.predict(xts)

In [56]:
from sklearn import metrics

In [57]:
# pred

array([0, 0, 1, ..., 1, 0, 0])

In [58]:
# yts.shape

(86872,)

In [59]:
# auc = metrics.roc_auc_score(yts, pred)
# print(auc)

0.6939766612770781


In [62]:
# clf = RandomForestClassifier(n_estimators=2000, n_jobs=-1, random_state=0)
# clf.fit(xtr, ytr)
# # pred2 = clf.predict(xts)

In [63]:
# auc = metrics.roc_auc_score(yts, pred2)
# print(auc)

0.6951547096982549


In [83]:
clf = RandomForestClassifier(n_estimators=200, n_jobs=-1, random_state=0)
clf.fit(xtr, ytr)
pred2 = clf.predict(xts)

In [84]:
auc = metrics.roc_auc_score(yts, pred2)
print(auc)

0.5339208472135634


In [107]:
# 1 hour ahead xts2
clf = RandomForestClassifier(n_estimators=200, n_jobs=-1, random_state=0)
clf.fit(xtr, ytr)
pred3 = clf.predict(xts2)
auc = metrics.roc_auc_score(yts2, pred3)
print(auc)

0.5833333333333334


In [142]:
# 3 hours ahead xts3 0.69 with 100 estimators
clf = RandomForestClassifier(n_estimators=500, n_jobs=-1, random_state=0)
clf.fit(xtr, ytr)
pred4 = clf.predict(xts3)
auc = metrics.roc_auc_score(yts3, pred4)
print(auc)

0.7276785714285714


In [143]:
pred_result = xts3
pred_result['Actual'] = yts3
pred_result['Predicted'] = pred4

/var/folders/4b/l73ksqn90ls5dx9x6n2k93xc0000gn/T/ipykernel_44292/1153909768.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pred_result['Actual'] = yts3
/var/folders/4b/l73ksqn90ls5dx9x6n2k93xc0000gn/T/ipykernel_44292/1153909768.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pred_result['Predicted'] = pred4


In [178]:
# 1 month ahead xts3 0.69 with 100 estimators
clf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=0)
clf.fit(xtr, ytr)
pred5 = clf.predict(xts4)
auc = metrics.roc_auc_score(yts4, pred5)
print(auc)

0.5229219918952618


In [179]:
pred_result = xts4.copy()
pred_result['Actual'] = yts4
pred_result['Predicted'] = pred5

In [194]:
# 1 year ahead xts5
clf = RandomForestClassifier(n_estimators=250, n_jobs=-1, random_state=0)
clf.fit(xtr, ytr)
pred6 = clf.predict(xts5)
auc = metrics.roc_auc_score(yts5, pred6)
print(auc)

0.5339991122869345


In [195]:
pred_result = xts5.copy()
pred_result['Actual'] = yts5
pred_result['Predicted'] = pred6
pred_result.head()

,Crypto,Close t-1,Change_close t-1,Actual,Predicted
DateTime,,,,,
2021-09-30 23:00:00,0,2.126,0.005201,0,0
2021-09-30 23:00:00,9,0.953,0.001050,1,0
2021-09-30 23:00:00,4,23.831,0.004045,1,1
2021-09-30 23:00:00,5,153.510,0.005173,0,1
2021-09-30 23:00:00,8,253.270,-0.000355,0,1


In [196]:
pred_result.shape

(86872, 5)

In [197]:
pred_result.tail(30)

,Crypto,Close t-1,Change_close t-1,Actual,Predicted
DateTime,,,,,
2022-09-30 21:00:00,9,0.49021,-0.015287,0,0
2022-09-30 21:00:00,7,0.11584,-0.011013,0,0
2022-09-30 21:00:00,0,0.43160,-0.006217,0,0
2022-09-30 21:00:00,2,27.76000,-0.007508,0,0
2022-09-30 21:00:00,5,54.02000,-0.006803,0,1
2022-09-30 21:00:00,8,147.36000,-0.006673,0,0
2022-09-30 21:00:00,6,0.06136,-0.004865,0,0
2022-09-30 21:00:00,1,19528.64000,-0.010912,0,0
2022-09-30 21:00:00,4,7.60600,-0.004711,0,1


In [ ]:
#Buid a naive baseline 